In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime as dt
from textblob import TextBlob
from nltk.corpus import stopwords

In [24]:
df = pd.read_csv("VacineCard_Nohas.csv")

In [27]:
def preprocess_tweets(tweet):
    
    fo = open("stopwords.txt", "r+")
    stop_words = list(fo.read().split(','))
    translation={39:None}
    processed_tweet = tweet
    processed_tweet=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())
    processed_tweet = " ".join(word for word in processed_tweet.split() if word not in str(stop_words).translate(translation))
    return(processed_tweet)
df['Processed Tweet'] = df['tweet'].apply(lambda x: preprocess_tweets(x.lower()))

In [28]:
#here we are making a new column 'polarity' by applying textblob function on the processed tweets which are in english language
from textblob import TextBlob
df['polarity'] = df[df['language']=='en']['Processed Tweet'].apply(lambda x: TextBlob(x).sentiment[0])

In [29]:
df['sentiment'] = df['polarity'].apply(lambda x: 'positive' if x > 0 else('negative' if x<0  else 'neutral'))

In [30]:
data = df[['tweet','sentiment']]
data

,tweet,sentiment
0,Here’s my vaccine card https://t.co/Zfb1P81jSE,neutral
1,"𝗙𝗼𝗿 𝗦𝗲𝗽𝘁. 𝟮𝟳, 𝟮𝟬𝟮𝟭 (𝗠𝗼𝗻𝗱𝗮𝘆), those who missed ...",positive
2,@GirlFuturist @brucewilson Arts degree with va...,positive
3,@chipfranklin @POTUS What exactly is a « vacci...,positive
4,@prairiecentrist In BC he wouldn't even be all...,neutral
...,...,...
20005,@Chilly_MIV Yeah ikr and I lost my vaccine car...,positive
20006,i hate to break it to you but buying a fake va...,negative
20007,@Martina Did you ask to see his vaccine card. ...,neutral
20008,"(cont’d) “Bring your vaccine card, picture ide...",neutral


In [54]:
data = data[data.sentiment != "neutral"]
data['tweet'] = data['tweet'].apply(lambda x: x.lower())
data['tweet'] = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'positive'].size)
print(data[ data['sentiment'] == 'negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_features = 2000
tokenizer = Tokenizer(nb_words=max_features, split=' ')
tokenizer.fit_on_texts(data['tweet'].values)
X = tokenizer.texts_to_sequences(data['tweet'].values)
X = pad_sequences(X)

16018
9470


In [71]:
X.shape[1]

61

In [73]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1], ))
model.add(LSTM(lstm_out))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 61, 128)           256000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [74]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 96)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(10195, 61) (10195, 2)
(2549, 61) (2549, 2)


In [60]:
# Our vectorized labels
y_train = np.asarray(train_labels).astype('float32').reshape((-1,1))
y_test = np.asarray(test_labels).astype('float32').reshape((-1,1))

NameError: name 'train_labels' is not defined

In [75]:
# batch_size = 32
model.fit(X_train, Y_train, epochs = 50, batch_size=32, verbose = 2)

# history=model.fit(X_train,Y_train,batch_size=80,epochs=6, validation_split=0.1)# here we are starting the training of model by feeding the training data
# print('Training finished !!')

Epoch 1/50
319/319 - 7s - loss: 0.4804 - accuracy: 0.7667
Epoch 2/50
319/319 - 2s - loss: 0.2654 - accuracy: 0.8953
Epoch 3/50
319/319 - 2s - loss: 0.1863 - accuracy: 0.9278
Epoch 4/50
319/319 - 2s - loss: 0.1423 - accuracy: 0.9459
Epoch 5/50
319/319 - 2s - loss: 0.1155 - accuracy: 0.9575
Epoch 6/50
319/319 - 2s - loss: 0.0960 - accuracy: 0.9639
Epoch 7/50
319/319 - 2s - loss: 0.0761 - accuracy: 0.9725
Epoch 8/50
319/319 - 2s - loss: 0.0702 - accuracy: 0.9765
Epoch 9/50
319/319 - 2s - loss: 0.0592 - accuracy: 0.9789
Epoch 10/50
319/319 - 2s - loss: 0.0416 - accuracy: 0.9857
Epoch 11/50
319/319 - 2s - loss: 0.0440 - accuracy: 0.9861
Epoch 12/50
319/319 - 2s - loss: 0.0341 - accuracy: 0.9895
Epoch 13/50
319/319 - 2s - loss: 0.0201 - accuracy: 0.9945
Epoch 14/50
319/319 - 2s - loss: 0.0089 - accuracy: 0.9974
Epoch 15/50
319/319 - 2s - loss: 0.0353 - accuracy: 0.9885
Epoch 16/50
319/319 - 2s - loss: 0.0218 - accuracy: 0.9922
Epoch 17/50
319/319 - 2s - loss: 0.0256 - accuracy: 0.9915
Epoch 

In [76]:

validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

33/33 - 1s - loss: 1.4869 - accuracy: 0.8913
score: 1.49
acc: 0.89


In [77]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1